# Enoda Technical Challenge

## 1. Exploratory Data Analysis

This dataset (https://zenodo.org/records/4549296) contains power measurements and meteorological forecasts relatvei to a set of 24 substation power meters from the distribution grid in Switzerland.

The power measurements are provided as a pickle dataset, which includes:

For each phase:

- mean active and reactive power
- voltage magnitude
- maximum total harmonic distortion (THD)
- voltage frequency 
- the average power over the three phases.
- The latter one has been used as target variable in the aforementioned paper.

The meteorological forecasts are provided as a Hierarchical Data Format 5 file, which includes:

- temperature
- global horizontal and normal irradiance (GHI and GNI, respectively)
- relative humidity (RH)
- pressure
- wind speed and direction

In [ ]:
from pathlib import Path
import pandas as pd

DATA_FLD = Path("data")

nwp_data = pd.read_hdf(DATA_FLD / "nwp_data.h5","df")
power_data = pd.read_pickle(DATA_FLD / "power_data.p")

In [ ]:
# reshape data
nwp_cols_to_keep = ["ghi_backwards", "windspeed", "temperature"]
nwp_df = pd.concat([
    pd.DataFrame(nwp_data[col].tolist(), index=nwp_data.index).add_prefix(f"{col}_")
    for col in nwp_cols_to_keep
], axis=1)

In [ ]:
power_data.keys()

In [ ]:
power_data_cols = ['freqA', 'P_mean', 'Q_mean', 'V_mean']

In [ ]:
power_df = pd.concat([power_data[col].add_suffix(f"_{col}") for col in power_data_cols], axis=1)

## 2. Feature Engineering
- Lags of power mean (previous hour, previous day)
- Moving average of power mean
- Hour of day, day of week, month
- Holiday or not

In [ ]:
import datetime as dt

holidays = [dt.datetime.strptime(i, "%d.%m.%Y").date() for i in pd.read_csv(DATA_FLD / "holidays.txt", delimiter="\t").squeeze().values]

In [ ]:
pd.plotting.autocorrelation_plot(power_df["S1_P_mean"].tail(6*24*30)) # last month

In [ ]:
lag_df = pd.concat([power_df["S1_P_mean"], power_df["S1_P_mean"].shift(24*6)], axis=1)
lag_df.columns = ["power", "power-24hr"]
lag_df.plot.scatter(x="power", y="power-24hr", alpha=0.1, s=0.2, grid=True, figsize=(3,3) ,title="-24hr Lag Correlation")

In [ ]:
ax=power_df["S1_P_mean"].groupby(power_df.index.hour).mean().plot(grid=True, figsize=(5, 3), title="Mean power")
ax.set_xlabel("Hour of Day")

In [ ]:
hour_month_df = power_df["S1_P_mean"].groupby([power_df.index.hour, power_df.index.month]).mean().unstack()
ax = hour_month_df.plot(grid=True, figsize=(5, 3), title="Mean power by hour by month")
ax.legend(loc='center left', bbox_to_anchor=(1.0, 0.5))
ax.set_xlabel("Hour of day")

In [ ]:
pd.concat([power_df["S1_P_mean"], nwp_df["temperature_0"]], axis=1).plot.scatter(y="S1_P_mean", x="temperature_0", alpha=0.05, s=0.1, grid=True, figsize=(3,3) ,title="Temperature vs Power Correlation")

In [ ]:
rolling_mean_df = pd.concat([power_df["S1_P_mean"], power_df["S1_P_mean"].shift(6).rolling(window="1h").mean()], axis=1)
rolling_mean_df.columns= ["Power", "1_hr_rolling_power"]

rolling_mean_df.plot.scatter(x="Power", y="1_hr_rolling_power", alpha=0.1, s=0.2, grid=True, figsize=(3,3) ,title="Hourly (previous) rolling mean Correlation")

In [ ]:
from typing import List

def create_lag_features_df(df: pd.DataFrame, lags: List[int]=[6, 12, 24, 24*7])->pd.DataFrame:
    lagged_frames = [
        df.shift(lag*6).add_prefix(f'lag{lag}hr_')
        for lag in lags
    ]
    return pd.concat(lagged_frames, axis=1)

In [ ]:
power_df.filter(like="P_mean").filter(like="S")

#### Select Target Column

In [ ]:
Y_COL = "S1_P_mean"

In [ ]:
#dataframes
power_rolling_mean = power_df.filter(like="P_mean").filter(like="S").shift(1).rolling(window="1h").mean().add_suffix("_rolling_mean")
lagged_df = create_lag_features_df(power_df.filter(like="P_mean").filter(like="S"))

#series
hour_of_day = pd.Series(power_df.index.hour)
day_of_week = pd.Series(power_df.index.dayofweek)
month = pd.Series(power_df.index.month)
is_holiday = [i in holidays for i in power_df.index.date]

extra_features_df = pd.DataFrame(
    {
        "hour_of_day":hour_of_day,
        "day_of_week":day_of_week,
        "month":month,
        "is_holiday":is_holiday,
    },
    index = power_df.index
)
 
features_df = pd.concat([nwp_df, extra_features_df, lagged_df, power_rolling_mean], axis=1) # join weather data, lagged power data and timeseries indicators

In [ ]:
selected_features = features_df.columns

## 3. Modelling
Use XGboost as a multi output regressor

In [ ]:
forecast_horizon = 6*24  # 24 hours ahead at 10-min intervals

# Create target columns
leads = range(1, forecast_horizon + 1)

lead_frames = [
        power_df[Y_COL].to_frame().shift(-lead).add_prefix(f'+{lead}_')
        for lead in leads
    ]

targets_df = pd.concat(lead_frames, axis=1).dropna(how="all", axis=1).dropna(how="any", axis=0)

In [ ]:
final_features_df = features_df[selected_features].loc[targets_df.index]

## 4. Modelling

## Train, test, validation split

In [ ]:
[i for i in final_features_df.columns]

In [ ]:
from sklearn.model_selection import train_test_split
 
X_temp, X_val, y_temp, y_val = train_test_split(final_features_df, targets_df, test_size=0.2, random_state=42, shuffle=False)# don't shuffle since timeseries
X_train, X_test, y_train, y_test = train_test_split(X_temp, y_temp, test_size=0.2, random_state=42, shuffle=False)# don't shuffle since timeseries

## Hyperparameter Tuning (Optuna)

In [ ]:
from sklearn.multioutput import MultiOutputRegressor
from lightgbm import LGBMRegressor
from sklearn.metrics import make_scorer, mean_squared_error
import numpy as np

# Define scoring
def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

scorer = make_scorer(rmse)

In [ ]:
# import optuna

# def objective(trial):
#     param = {
#         "n_estimators": trial.suggest_int("n_estimators", 50, 100),
#         "max_depth": trial.suggest_int("max_depth", -1, 5),
#         "num_leaves": trial.suggest_int("num_leaves", 20, 50),
#         "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3),
#         "random_state": 42
#     }

#     model = MultiOutputRegressor(LGBMRegressor(**param))
#     model.fit(X_train, y_train)

#     y_pred = model.predict(X_test)
#     rmse = rmse(y_test, y_pred))

#     return rmse

In [ ]:
# study = optuna.create_study()
# study.optimize(objective, n_trials=50)

In [ ]:
# from plotly.io import show

# fig = optuna.visualization.plot_optimization_history(study)
# show(fig)

In [ ]:
best_params = {'n_estimators': 93, 'max_depth': 0, 'num_leaves': 42, 'learning_rate': 0.06768919270718152}

## 5. Evaluation
Check final metrics by time horizon and feature importance

In [ ]:
Y_COL

In [ ]:
model = MultiOutputRegressor(LGBMRegressor(**best_params, random_state=42, verbose=-1))

In [ ]:
final_model = model.fit(X_train, y_train)

In [ ]:
y_pred = final_model.predict(X_val)

In [ ]:
rmse(y_val, y_pred)

In [ ]:
rmse_s = [np.sqrt(i) for i in mean_squared_error(y_val, y_pred, multioutput="raw_values")]

In [ ]:
from sklearn.metrics import mean_absolute_percentage_error as mape

f"{mape(y_val, y_pred):.1%}"

In [ ]:
mape_s = mape(y_val, y_pred, multioutput="raw_values")

In [ ]:
pd.DataFrame(
    {
        "RMSE": rmse_s,
        "MAPE": [i*100 for i in mape_s],
    },
    index=targets_df.columns
).plot(subplots=True, figsize=(15, 5), title="Metrics over the horizon (+10min to +24hr)")

In [ ]:
#Feature importance

In [ ]:
from lightgbm import plot_importance

ax = plot_importance(final_model.estimators_[5], max_num_features=10)
ax.set_title("+1hr Model Feature Importance")

In [ ]:
plot_importance(final_model.estimators_[-1], max_num_features=10)
ax.set_title("+24hr Model Feature Importance")

In [ ]:
#Plots (show short vs long forecasting)

In [ ]:
y_pred_df = pd.DataFrame(y_pred, index=y_val.index)
y_pred_df.columns = y_val.columns

In [ ]:
y_val[f"+144_{Y_COL}"].head(6*24*7).plot(label="true", figsize=(10, 2), title="+24hr Horizon")
y_pred_df[f"+144_{Y_COL}"].head(6*24*7).plot(label="predicted")

In [ ]:
y_val[f"+6_{Y_COL}"].head(6*24*7).plot(label="true", figsize=(10, 2), title="+1hr Horizon")
y_pred_df[f"+6_{Y_COL}"].head(6*24*7).plot(label="predicted")

In [ ]:
# Plot as single series

In [ ]:
y_pred_df2 = pd.DataFrame(y_pred, index=y_val.index)
y_pred_df2.columns = y_val.columns

ls = []

for idx, col in enumerate(y_pred_df2.columns):
    ser = y_pred_df2[col]
    shift = idx+1
    ser.index = ser.index + (shift*pd.Timedelta("10m"))
    ls.append(ser)

In [ ]:
plot_df = pd.concat(ls, axis=1).head(6*24*10)
ax = plot_df.plot(alpha=0.4, legend=False, figsize=(15,5), color="lightgrey")
power_df.loc[plot_df.index, Y_COL].plot(ax=ax, color="black")
ax.set_title("24hr Forecasting (all horizons) vs real data");

## 7. Physical Reasoning
Demonstrate how the forecast respects or violates physics
- V = IZ (we don't know Z?)
- S = IV (apparent power)
- AP = VIcos(theta) - we have frequency and so can work out phase angle? Check for PF > 1
- Conservation of energy

Show with plots. Select a single time horizon, say hour ahead.

In [ ]:
power_data.keys()

In [ ]:
predicted_power_1hr_ser = pd.concat(ls, axis=1)["+6_S1_P_mean"]

In [ ]:
power_data_df = power_df.loc[predicted_power_1hr_ser.index]

In [ ]:
power_data_df = power_data_df.assign(
    **{"S1_S_mean_calc": lambda d: np.sqrt(d["S1_P_mean"]**2 + d["S1_Q_mean"]**2)}
).assign(
    **{"S1_I_mean_calc": lambda d: d["S1_S_mean_calc"]/d["S1_V_mean"]}
).assign(
     **{"S1_Z_mean_calc": lambda d: d["S1_V_mean"]/d["S1_I_mean_calc"]}
)

#### Check predicted active power doesn't exceed apparent power

In [ ]:
pd.concat([power_data_df["S1_S_mean_calc"], predicted_power_1hr_ser], axis=1).assign(flag=lambda d: d["+6_S1_P_mean"] > d["S1_S_mean_calc"]).astype({"flag": float}).head(6*24*7).plot(subplots=True, figsize=(20, 5), title="Predicted Active Power exceeds real Apparent Power")

#### Check Power Factor

In [ ]:
pd.concat(
    [power_data_df["S1_S_mean_calc"], 
     predicted_power_1hr_ser]
    , axis=1
).assign(
    pred_PF=lambda d: d["+6_S1_P_mean"] / d["S1_S_mean_calc"]
).assign(
    flag=lambda d: ~d["pred_PF"].between(0,1)
).astype(
    {"flag": float}
).head(6*24*7).plot(subplots=True, figsize=(20, 5), title="Predicted Active Power Leads to Non-physical Power Factor")

#### Phase angle check?

In [ ]:
pd.concat(
    [
        power_data_df["S1_S_mean_calc"],
        predicted_power_1hr_ser],
    axis=1
).assign(
    **{"S1_Theta_from_predicted_p+6": lambda d: np.rad2deg(np.arccos(d["+6_S1_P_mean"]/d["S1_S_mean_calc"]))}
).plot(subplots=True, figsize=(15, 5))

## 8. Anomaly Detection (extra)
In the historic data identify:
- voltage sags or spikes: sudden drops/spikes in voltage (try basic rolling std thresholding) / show frequency and THD for context
- overload conditions: 
- phase imbalance or anomalous switching: compare power/voltage across phases and look for outlier (compare max of phases to median of phases)
- look for spikes in power i.e show freq and tempeature for context

In [ ]:
power_data.keys()

In [ ]:
all_power_df = pd.concat([power_data[col].add_suffix(f"_{col}") for col in power_data.keys()], axis=1)

In [ ]:
v_lims = (220, 240) # legal limits

In [ ]:
all_power_df.filter(like="avgVmagA").apply(lambda d: ~d.between(*v_lims)).sum(axis=0)

In [ ]:
selected_meter = "S2"

In [ ]:
meter_df = all_power_df.filter(like=selected_meter)
meter_df.columns = [i.split("_", 1)[-1] for i in meter_df.columns]

In [ ]:
def flags_by_zscore(ser: pd.Series, window: str = "1hr", zscore:float = 2)->pd.Series:
    flags_ser = abs(ser - ser.rolling(window="1h").mean())/ ser.rolling(window="1h").std()>zscore
    flags_ser.name = "flag"
    return flags_ser.astype(float)

v_flags = flags_by_zscore(meter_df["V_mean"])

In [ ]:
v_df = pd.concat([meter_df["V_mean"], meter_df["V_mean"].rolling("1h").mean(), meter_df["V_mean"].rolling("1h").std(), abs(meter_df["V_mean"] -meter_df["V_mean"].rolling(window="1h").mean())/ meter_df["V_mean"].rolling(window="1h").std(), v_flags], axis=1).tail(500)
v_df.columns = ["10min_avg", "1hr_avg", "1hr_std", "Z_score","flag"]
v_df.plot(figsize=(15, 4), subplots=True, title="Hourly Voltage Anomalies flagged by Z score > 2")

In [ ]:
meter_df = all_power_df.filter(like="5e9c55269b890ad82c8ebbd146ea2a563fe768ce")
meter_df.columns = [i.split("_", 1)[-1] for i in meter_df.columns]

In [ ]:
meter_df.head(3)

In [ ]:
import matplotlib.pyplot as plt

def flags_by_limits(ser:pd.Series, lower: float = 49.9, upper: float = 50.1)->pd.Series:
    return ser.between(lower, upper).astype(float).rename("flag")

pd.concat([meter_df["PA"], meter_df["freqA"], flags_by_limits(meter_df["freqA"])], axis=1).iloc[4200:5300].plot(subplots=True, figsize=(15,3), title="Frequency drop potentially indicating overload")

In [ ]:
ax=meter_df["freqA"].hist(bins=20, figsize=(4,3))
ax.set_title("Frequency Distribution (nominal 50 Hz)")

In [ ]:
meter_df[["avgVmagA", "avgVmagB", "avgVmagC"]].iloc[3500:4000].plot(figsize=(15,2), alpha=0.5)

In [ ]:
phase_df = meter_df[["PA", "PB", "PC"]].assign(phase_min=meter_df[["PA", "PB", "PC"]].min(axis=1)).assign(phase_max=meter_df[["PA", "PB", "PC"]].max(axis=1)).assign(phase_imbalance=lambda d: d["phase_max"]-d["phase_min"]).drop(columns=["phase_min", "phase_max"])

In [ ]:
phase_df["phase_imbalance"].hist(bins=50)

In [ ]:
phase_df[["PA", "PB", "PC"]].iloc[3500:4000].plot(figsize=(15,2), alpha=0.5)

In [ ]:
phase_df["phase_imbalance"].iloc[3500:4000].plot(figsize=(15,1), alpha=0.5, title="Phase imbalance (power)")

In [ ]:
flags_by_limits(phase_df["phase_imbalance"].iloc[3500:4000], 0, 25).mul(-1).add(1).plot(figsize=(15,1), alpha=0.5, title="Threshold flags")